In [22]:
from pronto import Ontology
import json
import pandas as pd

In [23]:
do = Ontology('https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/master/src/ontology/HumanDO.obo')

In [24]:
def read_term (i):
    ii = str(i).replace('Term(','').replace(')','').replace("'",'').split(',')
    doid = ii[0].replace('DOID:','')
    return doid
    

In [25]:
def read_xref (j):
    jj = set(j)
    for n in jj:
        xref = str(n).replace('Xref','').replace('(','').replace(')','').replace("'",'').split(':')
        if 'UMLS_CUI' in xref[0]:
            umls = xref[1]
            return umls

In [26]:
def do_umls(do):
    mapp = {}
    for i in do.terms():
        doid = read_term(i)
        umls = read_xref(i.xrefs)
        if umls is not None:
            mapp[umls] = doid
    return mapp


In [27]:
do2umls = do_umls(do)
with open('do2umls.json','w') as f:
    json.dump(do2umls,f)

In [28]:
repo = pd.read_csv('/home/lxu/Downloads/full.csv')

In [29]:
repo.head()

,drug_name,drug_id,ind_name,ind_id,NCT,status,phase,DetailedStatus
0,Lepirudin,DB00001,Heparin-induced thrombocytopenia with thrombosis,C0272275,NaN,Approved,NaN,NaN
1,Cetuximab,DB00002,Squamous cell carcinoma of mouth,C0585362,NaN,Approved,NaN,NaN
2,Cetuximab,DB00002,Squamous cell carcinoma of nose,C3163899,NaN,Approved,NaN,NaN
3,Cetuximab,DB00002,Squamous cell carcinoma of pharynx,C1319317,NaN,Approved,NaN,NaN
4,Cetuximab,DB00002,Laryngeal Squamous Cell Carcinoma,C0280324,NaN,Approved,NaN,NaN


In [30]:
doid = []
for _,row in repo[['ind_id']].iterrows():
    if row['ind_id'] in do2umls.keys():
        
        do_id = do2umls[row['ind_id']]
        
        doid.append(do_id)
    else:
        doid.append(None)
    


In [31]:
repo.insert(4,'do_id',doid)


In [32]:
nodes = pd.read_csv('/home/lxu/Downloads/integrate-master/data/nodes.tsv','\t')

In [33]:
compound = nodes.loc[nodes['kind']=='Compound']['id'].tolist()
disease = nodes.loc[nodes['kind']=='Disease']['id'].tolist()

In [34]:
drug_in_hetio = []
disease_in_hetio = []
for _, row in repo[['drug_id','do_id']].iterrows():
    if 'Compound::'+row['drug_id'] in compound:
        drug_in_hetio.append(1)
    else:
        drug_in_hetio.append(0)
    if row['do_id'] and 'Disease::DOID:'+row['do_id'] in disease:
        disease_in_hetio.append(1)
    else:
        disease_in_hetio.append(0)
    

In [35]:
disease_in_hetio.count(1)

729

In [36]:
repo.insert(2,'drug_in_hetio',drug_in_hetio)
repo.insert(5,'disease_in_hetio',disease_in_hetio)

In [37]:
repo.head()

,drug_name,drug_id,drug_in_hetio,ind_name,ind_id,disease_in_hetio,do_id,NCT,status,phase,DetailedStatus
0,Lepirudin,DB00001,0,Heparin-induced thrombocytopenia with thrombosis,C0272275,0,None,NaN,Approved,NaN,NaN
1,Cetuximab,DB00002,0,Squamous cell carcinoma of mouth,C0585362,0,None,NaN,Approved,NaN,NaN
2,Cetuximab,DB00002,0,Squamous cell carcinoma of nose,C3163899,0,None,NaN,Approved,NaN,NaN
3,Cetuximab,DB00002,0,Squamous cell carcinoma of pharynx,C1319317,0,None,NaN,Approved,NaN,NaN
4,Cetuximab,DB00002,0,Laryngeal Squamous Cell Carcinoma,C0280324,0,2876,NaN,Approved,NaN,NaN


In [38]:
repo_hetio = []
for _,row in repo[['drug_id','drug_in_hetio','disease_in_hetio','do_id','status']].iterrows():
    if row['drug_in_hetio'] ==1 and row['disease_in_hetio'] == 1:
        if row['status'] == 'Approved':
            drug = 'Compound::' + row['drug_id']
            disease = 'Disease::DOID:' + row['do_id']
            repo_hetio.append((drug,disease,1))
        else:
            repo_hetio.append((drug,disease,0))

In [46]:
df = pd.DataFrame(repo_hetio, columns = ['drug','disease','status'])
df
df.to_csv('repo_data.csv')

In [40]:
df_pos = df.loc[df['status']==1]
df_neg = df.loc[df['status']==0]

In [41]:
len(df_pos)

415

In [42]:
len(df_neg)

195

In [47]:
repo_data = pd.read_csv('repo_data.csv')

In [48]:
repo_data

,Unnamed: 0,drug,disease,status
0,0,Compound::DB00091,Disease::DOID:7148,1
1,1,Compound::DB00137,Disease::DOID:10871,1
2,2,Compound::DB00153,Disease::DOID:11476,1
3,3,Compound::DB00169,Disease::DOID:11476,1
4,4,Compound::DB00177,Disease::DOID:10763,1
5,5,Compound::DB00178,Disease::DOID:10763,1
6,6,Compound::DB00180,Disease::DOID:2841,1
7,7,Compound::DB00182,Disease::DOID:9970,1
8,8,Compound::DB00182,Disease::DOID:8986,1
9,9,Compound::DB00184,Disease::DOID:0050742,1


In [49]:
from urllib.request import urlretrieve

In [51]:
urlretrieve('https://github.com/dhimmel/learn/blob/master/prediction/features/features.tsv.bz2?raw=true','/home/lxu/Downloads/bz2try.bz2')

('/home/lxu/Downloads/bz2try.bz2', <http.client.HTTPMessage at 0x7f867f8f31d0>)